In [2]:
import requests
import re
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
from io import StringIO  
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime

In [5]:
url = 'https://www.baseball-reference.com/leagues/majors/2018-standard-batting.shtml'
# url_league = 'https://www.baseball-reference.com/players/l/lindofr01.shtml'
r = requests.get(url)
r.status_code

200

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')
soup.title.string

'2018 Major League Baseball Standard Batting | Baseball-Reference.com'